In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter

from lifetimes.plotting import *
from lifetimes.utils import *
import joblib

import plotly.graph_objects as go


In [2]:
df = pd.read_csv("Data/cleaned_sales_data")
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)
df.drop(df.loc[df["Customer ID"] == 12346.0].index, inplace = True)
#df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1039537 entries, 0 to 1039561
Data columns (total 10 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1039537 non-null  int64         
 1   StockCode    1039537 non-null  int64         
 2   Description  1037881 non-null  object        
 3   Quantity     1039537 non-null  int64         
 4   InvoiceDate  1039537 non-null  datetime64[ns]
 5   Price        1039537 non-null  float64       
 6   Customer ID  802668 non-null   float64       
 7   Country      1039537 non-null  object        
 8   Revenue      1039537 non-null  float64       
 9   refund       0 non-null        float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(2)
memory usage: 87.2+ MB


In [13]:
today_date = dt.datetime(2011, 12, 12)

rfm = df.groupby("Customer ID").agg({"InvoiceDate": [lambda date: (date.max() - date.min()).days,
                                                     lambda date: (today_date - date.min()).days],
                                     "Invoice": lambda num: num.nunique(),
                                      "Revenue": lambda price: price.sum()}) #total price per customer

rfm.columns = rfm.columns.droplevel(0)
rfm.columns = ['recency', "T", 'frequency', "monetary_value"]

# Calculating average monetary values per order:
rfm["monetary_value"] = rfm["monetary_value"] / rfm["frequency"]

In [14]:
rfm

,recency,T,frequency,monetary_value
Customer ID,,,,
12347.0,402,406,8,704.165000
12348.0,362,440,5,331.680000
12349.0,570,591,3,1226.230000
12350.0,0,312,1,294.400000
12351.0,0,377,1,300.930000
...,...,...,...,...
18283.0,654,660,22,124.122727
18284.0,0,433,1,411.680000
18285.0,0,662,1,377.000000


In [17]:
bgf = BetaGeoFitter(penalizer_coef=0.01)
bgf.fit(rfm['frequency'], rfm['recency'], rfm['T'])

rfm["probability_alive"] = bgf.conditional_probability_alive(rfm['frequency'], rfm['recency'], rfm['T'])
rfm

,recency,T,frequency,monetary_value,probability_alive
Customer ID,,,,,
12347.0,402,406,8,704.165000,0.971673
12348.0,362,440,5,331.680000,0.880941
12349.0,570,591,3,1226.230000,0.916467
12350.0,0,312,1,294.400000,0.015541
12351.0,0,377,1,300.930000,0.010767
...,...,...,...,...,...
18283.0,654,660,22,124.122727,0.988626
18284.0,0,433,1,411.680000,0.008196
18285.0,0,662,1,377.000000,0.003488


In [26]:
# Prediction of expected number of transaction for each customer for one year (365 days)
rfm['expctd_num_of_purch'] = bgf.predict(90, rfm['frequency'], rfm['recency'], rfm['T']) 
rfm.sort_values("expctd_num_of_purch",ascending=False).head()

,Customer ID,recency,T,frequency,monetary_value,probability_alive,expctd_num_of_purch,CLV
1824,14911,737,740,373,741.701367,0.997664,43.101961,116259.898846
273,12748,734,737,322,162.084814,0.997770,37.374270,22222.129309
3950,17841,735,739,211,335.767630,0.997100,24.455944,29952.121691
2112,15311,738,740,207,562.686763,0.998317,23.993238,49073.402728
513,13089,734,739,203,575.063350,0.996228,23.513032,49139.225493


In [27]:
rfm.monetary_value.astype(int)
rfm = rfm[rfm.monetary_value >=1]
rfm = rfm[rfm['frequency']>1]

ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(rfm['frequency'],
        rfm['monetary_value'])
print(ggf)

<lifetimes.GammaGammaFitter: fitted with 4233 subjects, p: 1.83, q: 3.96, v: 613.20>


In [28]:

rfm["CLV"] = ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    rfm['frequency'],
    rfm['recency'],
    rfm['T'],
    rfm['monetary_value'],
    time=12, # months
    discount_rate=0.01)

In [29]:
rfm.reset_index(inplace = True)


In [30]:
rfm["Customer ID"] = rfm["Customer ID"].astype(int)
rfm.to_csv("Data/rfm", index = False)
rfm

,index,Customer ID,recency,T,frequency,monetary_value,probability_alive,expctd_num_of_purch,CLV
0,0,12347,402,406,8,704.165000,0.971673,1.790380,4145.297084
1,1,12348,362,440,5,331.680000,0.880941,1.011752,1238.190811
2,2,12349,570,591,3,1226.230000,0.916467,0.539137,1797.787902
3,3,12352,356,394,9,192.171111,0.942280,1.980055,1556.670517
4,4,12353,204,410,2,203.380000,0.516001,0.321745,319.769947
...,...,...,...,...,...,...,...,...,...
4228,4228,18281,397,579,2,100.570000,0.717020,0.325524,261.741813
4229,4229,18282,118,128,2,89.025000,0.862802,1.435056,1049.453999
4230,4230,18283,654,660,22,124.122727,0.988626,2.941923,1511.545892
4231,4231,18286,247,725,2,623.215000,0.249516,0.091742,169.733555


In [31]:
rfm.describe()

,index,Customer ID,recency,T,frequency,monetary_value,probability_alive,expctd_num_of_purch,CLV
count,4233.000000,4233.000000,4233.000000,4233.000000,4233.000000,4233.000000,4.233000e+03,4.233000e+03,4.233000e+03
mean,2116.000000,15317.920624,377.943303,522.118356,8.262462,401.196866,6.864230e-01,1.291675e+00,2.124527e+03
std,1222.106174,1697.183067,229.931600,203.626342,14.498382,1375.891308,3.477336e-01,1.869148e+00,8.004604e+03
min,0.000000,12347.000000,0.000000,4.000000,2.000000,3.725000,3.170615e-43,5.745758e-42,7.647434e-39
25%,1058.000000,13859.000000,165.000000,406.000000,3.000000,198.435000,4.461037e-01,3.107160e-01,3.308227e+02
50%,2116.000000,15321.000000,382.000000,586.000000,5.000000,299.500000,8.692642e-01,8.365628e-01,9.504181e+02
75%,3174.000000,16784.000000,588.000000,689.000000,9.000000,432.800000,9.515620e-01,1.619426e+00,2.113894e+03
max,4232.000000,18287.000000,738.000000,740.000000,373.000000,84236.250000,9.983172e-01,4.310196e+01,2.550548e+05


rfm["clv_bins"] = pd.qcut(rfm['CLV'], q=4, labels=[1, 2, 3, 4])
rfm


plt.figure(figsize=(20,10))

sns.scatterplot(x = "frequency", y = "recency", data=rfm, hue = "clv_bins");

In [24]:
rfm[rfm.probability_alive <0.2]

,Customer ID,recency,T,frequency,monetary_value,probability_alive,expctd_num_of_purch,CLV
15,12369,228,643,3,503.716667,0.193672,0.408655,174.242342
22,12376,28,419,2,191.575000,0.012655,0.029430,7.518022
30,12386,29,368,2,330.400000,0.019347,0.050303,16.434505
52,12419,27,415,2,195.505000,0.012334,0.028922,7.446760
69,12439,43,727,2,579.090000,0.005070,0.007220,3.276714
...,...,...,...,...,...,...,...,...
4205,18247,216,633,3,279.910000,0.173482,0.371285,108.276106
4211,18258,108,574,6,343.278333,0.000741,0.003009,0.978328
4213,18260,551,726,23,433.135217,0.162705,1.812664,721.322325
4220,18271,121,733,2,340.425000,0.045905,0.064891,21.512969
